## Второй бот для отправки уведомлений о встречах

In [ ]:
# pip install schedule

In [1]:
import sqlite3
import telebot
from telebot import types
import json
import random
from datetime import datetime, date, timedelta
from time import sleep
import pandas as pd

import schedule 

try:
    conn = sqlite3.connect('meeting_reminder.db')

    # курсор для работы с таблицами
    cursor = conn.cursor()

    # cursor.execute("SELECT * FROM meeting_reminder WHERE chat_id=={}".format(message.chat.id))
    cursor.execute("SELECT * FROM meeting_reminder")
    lst_all_meets = cursor.fetchall()
    lst_meet_text_msg_new = []

    for meet in lst_all_meets:
        print(meet)
        meet_time = meet[2]
        meet_location = meet[3]
        meet_name = meet[4]
        meet_schedule = meet[5]
#             print(f'meet_time={meet_time}, meet_location={meet_location}, meet_name={meet_name}, meet_schedule={meet_schedule}, ') 

        try:
            #каждый день по будням
            if meet_schedule == 'daily on weekdays' or meet_schedule == 'every day on weekdays':
                lst_meet_text_msg_new.append(f"schedule.every().monday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")
                lst_meet_text_msg_new.append(f"schedule.every().tuesday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")
                lst_meet_text_msg_new.append(f"schedule.every().wednesday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")
                lst_meet_text_msg_new.append(f"schedule.every().thursday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")
                lst_meet_text_msg_new.append(f"schedule.every().friday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

            #каждый день
            elif meet_schedule == 'every day':
                lst_meet_text_msg_new.append(f"schedule.every().day.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")


            #каждый какой-то день недели
            elif meet_schedule == 'every monday':
                lst_meet_text_msg_new.append(f"schedule.every().monday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

            elif meet_schedule == 'every tuesday':
                lst_meet_text_msg_new.append(f"schedule.every().tuesday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

            elif meet_schedule == 'every wednesday':
                lst_meet_text_msg_new.append(f"schedule.every().wednesday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

            elif meet_schedule == 'every thursday':
                lst_meet_text_msg_new.append(f"schedule.every().thursday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

            elif meet_schedule == 'every friday':
                lst_meet_text_msg_new.append(f"schedule.every().friday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

            elif meet_schedule == 'every saturday':
                lst_meet_text_msg_new.append(f"schedule.every().saturday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

            elif meet_schedule == 'every sunday':
                lst_meet_text_msg_new.append(f"schedule.every().sunday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

            # в один определенный день
            else:
                try:
                    if date.fromisoformat(meet_schedule) == date.today():
                        lst_meet_text_msg_new.append(f"schedule.every().day.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")
                except:
                    print(f'Error: meet_schedule not ISO format: YYYY-MM-DD {meet}')
        except:
            print('Error: somthing wrong with scheduled')
except: 
    print('Error while parse table')


(1, -4073737804, '23:15', 'home', 'Событие "Что-то важное"', 'every day', '23:15;home;Что-то важное;every day')


In [5]:
lst_meet_text_msg_new

['schedule.every().day.at(\'23:15\').do(dop_send_stat, message, \'23:15\', \'home\', \'Событие "Что-то важное"\')']

In [3]:
import sqlite3
import telebot
from telebot import types
import json
import random
from datetime import datetime, date, timedelta
from time import sleep
import pandas as pd

import schedule 
# https://schedule.readthedocs.io/en/stable/reference.html 
# https://schedule.readthedocs.io/en/stable/examples.html
# https://schedule.readthedocs.io/en/stable/_modules/schedule.html#run_pending - код библиотеки


bot = telebot.TeleBot("6637092297:AAE7hUz7mBorW-gP9r_qL_x-3UdEWlPRBpU") # нужен токен вашего бота meetingreminderzi16_notify_bot


# Функция для отправки сообщения с инфой
def dop_send_stat(message, meet_time, meet_location, meet_name):
    bot.send_message(message.chat.id, f'{meet_name} в {meet_time}: \n{meet_location}')
    


# Функция перекладывает данные из таблицы в датафрейм и формирует новое поле для установки на расписание    
def parse_table_meet_remind(message):
    
    try:
        conn = sqlite3.connect('meeting_reminder.db')

        # курсор для работы с таблицами
        cursor = conn.cursor()

        # cursor.execute("SELECT * FROM meeting_reminder WHERE chat_id=={}".format(message.chat.id))
        cursor.execute("SELECT * FROM meeting_reminder")
        lst_all_meets = cursor.fetchall()

        lst_meet_text_msg_new = []

        for meet in lst_all_meets:
            meet_time = meet[2]
            meet_location = meet[3]
            meet_name = meet[4]
            meet_schedule = meet[5]
#             print(f'meet_time={meet_time}, meet_location={meet_location}, meet_name={meet_name}, meet_schedule={meet_schedule}, ') 

            try:
                #каждый день по будням
                if meet_schedule == 'every day on weekdays':
                    lst_meet_text_msg_new.append(f"schedule.every().monday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")
                    lst_meet_text_msg_new.append(f"schedule.every().tuesday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")
                    lst_meet_text_msg_new.append(f"schedule.every().wednesday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")
                    lst_meet_text_msg_new.append(f"schedule.every().thursday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")
                    lst_meet_text_msg_new.append(f"schedule.every().friday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

                #каждый день
                elif meet_schedule == 'every day':
                    lst_meet_text_msg_new.append(f"schedule.every().day.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")


                #каждый какой-то день недели
                elif meet_schedule == 'every monday':
                    lst_meet_text_msg_new.append(f"schedule.every().monday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

                elif meet_schedule == 'every tuesday':
                    lst_meet_text_msg_new.append(f"schedule.every().tuesday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

                elif meet_schedule == 'every wednesday':
                    lst_meet_text_msg_new.append(f"schedule.every().wednesday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

                elif meet_schedule == 'every thursday':
                    lst_meet_text_msg_new.append(f"schedule.every().thursday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

                elif meet_schedule == 'every friday':
                    lst_meet_text_msg_new.append(f"schedule.every().friday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

                elif meet_schedule == 'every saturday':
                    lst_meet_text_msg_new.append(f"schedule.every().saturday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

                elif meet_schedule == 'every sunday':
                    lst_meet_text_msg_new.append(f"schedule.every().sunday.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")

                # в один определенный день
                else:
                    try:
                        if date.fromisoformat(meet_schedule) == date.today():
                            lst_meet_text_msg_new.append(f"schedule.every().day.at('{meet_time}').do(dop_send_stat, message, '{meet_time}', '{meet_location}', '{meet_name}')")
                    except:
                        print('Error: meet_schedule not ISO format: YYYY-MM-DD')
            except:
                print('Error: somthing wrong with scheduled')
    except: 
        print('Error while parse table')
                                         
    return lst_meet_text_msg_new

 

            
# шедулер для отправки сообщений в чат
@bot.message_handler(func = lambda message: 1==1)
def send_remind_message(message):
    # Собираю все встречи на сегодня вношу их в расписание

    while True:

        print(f'============================== {datetime.now()} =====================================')

        lst_meet_text_msg_new = parse_table_meet_remind(message)
        

        #Установка на расписание
        list_dags = []

        #Очистка расписания
        schedule.clear()

        #Задание нового расписания
        for schedule_text in lst_meet_text_msg_new:
    #             print(f'schedule_text = {schedule_text}')
            list_dags.append(eval(schedule_text))
#         print(list_dags)
        print('------------- я почти в цикле')
    
        i = 0
        #Запускаю отправлятор сообщений
        while i <= 3:
            print(f'------------- i = {i}')
            try:
                schedule.run_pending()
            except:
                logger.exception('Scheduler Exception') 
            sleep(5)
            
            i += 1
            
       
          
        

In [ ]:
# df_meet_remind = pd.DataFrame()

bot.polling(none_stop=True)

============================== 2024-02-11 23:56:35.643599 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-11 23:56:55.652422 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-11 23:57:16.193754 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-11 23:57:36.197373 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-11 23:57:56.202647 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
----------

------------- i = 2
------------- i = 3
============================== 2024-02-12 00:09:56.803761 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-12 00:10:16.808596 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-12 00:10:36.812225 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-12 00:10:56.818841 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-12 00:11:16.824265 =====================================
------------- я почти в цикле
----------

------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-12 00:23:17.014513 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-12 00:23:37.018362 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-12 00:23:57.022730 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-12 00:24:17.028475 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-12 00:24:37.034000 =====================================
------------- я почт

------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-12 00:36:37.669933 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-12 00:36:57.676039 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-12 00:37:17.682229 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-12 00:37:37.687848 =====================================
------------- я почти в цикле
------------- i = 0
------------- i = 1
------------- i = 2
------------- i = 3
============================== 2024-02-12 00:37:57.691078 =====================================
------------- я почт

In [34]:
str(schedule.get_jobs()[0]) #.next_run()

# for 
#     schedule.get_jobs()[0].next_run

lst_sched = schedule.get_jobs()
my_birthday = my_birthday.replace(year=today.year + 1)

In [80]:
lst_sched = schedule.get_jobs()

for my_dag in lst_sched:
    if datetime.now() <= my_dag.next_run \
          and date.today() + timedelta(days = 1) > date(my_dag.next_run.year, my_dag.next_run.month, my_dag.next_run.day):
        print(my_dag)
    

Job(interval=1, unit=days, do=dop_send_stat, args=(<telebot.types.Message object at 0x0000014925832790>, '15:20', 'https://dion.vc/event/skonkin', 'Daily'), kwargs={})
Job(interval=1, unit=days, do=dop_send_stat, args=(<telebot.types.Message object at 0x0000014925832790>, '15:21', 'https://dion.vc/event/skonkin', 'Daily'), kwargs={})
Job(interval=1, unit=days, do=dop_send_stat, args=(<telebot.types.Message object at 0x0000014925832790>, '15:22', 'https://dion.vc/event/skonkin', 'Daily'), kwargs={})
Job(interval=1, unit=days, do=dop_send_stat, args=(<telebot.types.Message object at 0x0000014925832790>, '15:23', 'https://dion.vc/event/skonkin', 'Daily'), kwargs={})
Job(interval=1, unit=days, do=dop_send_stat, args=(<telebot.types.Message object at 0x0000014925832790>, '15:25', 'https://dion.vc/event/skonkin', 'Daily'), kwargs={})
Job(interval=1, unit=days, do=dop_send_stat, args=(<telebot.types.Message object at 0x0000014925832790>, '15:27', 'https://dion.vc/event/skonkin', 'Daily'), kwa

In [51]:
datetime.now()

datetime.datetime(2024, 2, 11, 18, 0, 5, 693519)

============================== 2024-02-11 18:00:06.285750 =====================================
[Every 1 day at 15:20:00 do dop_send_stat(<telebot.types.Message object at 0x0000014925832790>, '15:20', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-12 15:20:00), Every 1 day at 15:21:00 do dop_send_stat(<telebot.types.Message object at 0x0000014925832790>, '15:21', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-12 15:21:00), Every 1 day at 15:22:00 do dop_send_stat(<telebot.types.Message object at 0x0000014925832790>, '15:22', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-12 15:22:00), Every 1 day at 15:23:00 do dop_send_stat(<telebot.types.Message object at 0x0000014925832790>, '15:23', 'https://dion.vc/event/skonkin', 'Daily') (last run: [never], next run: 2024-02-12 15:23:00), Every 1 day at 15:25:00 do dop_send_stat(<telebot.types.Message object at 0x0000014925832790>, '15:25', 'https:

In [8]:
print('1')

1


In [ ]:
# подключаем базу данных
conn = sqlite3.connect('meeting_reminder.db')
# курсор для работы с таблицами
cursor = conn.cursor()

df_meet_remind = pd.read_sql_query("SELECT * FROM meeting_reminder", conn)
df_meet_remind['meet_text_msg_new'] = df_meet_remind.apply(fun_meet_text_msg_new, axis=1)

lst_meet_text_msg_new = list(df_meet_remind['meet_text_msg_new'])

 

In [13]:
def printhello():
    print('Hello')

In [20]:
schedule.every().day.at('20:02').do(printhello)
while True:
    try:
        schedule.run_pending()
    except:
        logger.exception('Scheduler Exception') 
    sleep(5)
           

KeyboardInterrupt: 

In [ ]:
# time_meet = datetime.strptime('1900-01-01 22:25:00', '%Y-%m-%d %H:%M:%S').replace(year=1900, month=1, day=1, microsecond=0)
# time_meet

In [ ]:
# datenow = datetime.now().replace(year=1900, month=1, day=1, microsecond=0)
# datenow

In [ ]:
# time_meet-datenow

In [1]:
#Первая версия сырая совсем
import sqlite3
import telebot
from telebot import types
import json
import random
from datetime import datetime
from time import sleep


bot = telebot.TeleBot("6637092297:AAE7hUz7mBorW-gP9r_qL_x-3UdEWlPRBpU") # нужен токен вашего бота meetingreminderzi16_notify_bot

# шедулер для отправки сообщений в чат
@bot.message_handler(func = lambda message: 1==1)
def send_remind_message(message):
    # Собираю все встречи на сегодня в ношу их в расписание (думаю с 8:00 до 19:00 нужно делать это каждые 10 минут, в остальное время только в 07:00)
    while True:
        if datetime.now().hour == 7:
            
    
    # 
    
    
    time_meet = datetime.strptime('1900-01-01 14:28', '%Y-%m-%d %H:%M').replace(year=1900, month=1, day=1, microsecond=0)
    print(f'time_meet = {time_meet}')
    while True:
        time_diff = time_meet - datetime.now().replace(year=1900, month=1, day=1, microsecond=0)
        datenow = datetime.now().replace(year=1900, month=1, day=1, microsecond=0)
        print(time_meet - datetime.now().replace(year=1900, month=1, day=1, microsecond=0))
#         print(f'time_meet = {time_meet}')
        print(f'datenow={datenow}')
        print(f'time_diff={time_diff}')
        if time_diff.days >= 0 and time_diff.seconds >=0 and time_diff.seconds <= 120:
            text = f'Встреча начнется через: {str(time_diff)} секунд'
            break
        elif time_diff.days < 0:
            text = f'Встреча уже началась: {str(-time_diff)} секунд назад'
            break
            
        sleep(10)
#     text = 'ha'    
    bot.send_message(message.chat.id, text)
    print('отправили')

    


IndentationError: expected an indented block after 'if' statement on line 17 (3214300635.py, line 23)

In [ ]:
до 20240211
# Функция перекладывает данные из таблицы в датафрейм и формирует новое поле для установки на расписание    
def parse_table_meet_remind(message):
    # подключаем базу данных
    conn = sqlite3.connect('meeting_reminder.db')
    
    # курсор для работы с таблицами
    cursor = conn.cursor()
    
    df_meet_remind = pd.read_sql_query("SELECT * FROM meeting_reminder WHERE chat_id=={}".format(message.chat.id), conn)
    df_meet_remind['meet_text_msg_new'] = df_meet_remind.apply(fun_meet_text_msg_new, axis=1)

    lst_meet_text_msg_new = list(df_meet_remind['meet_text_msg_new'])
    return lst_meet_text_msg_new
